[View in Colaboratory](https://colab.research.google.com/github/zacateras/nn-nbirds/blob/master/report_colab.ipynb)

In [ ]:
# If python3 is not loaded kill the hypervisor
# ! kill -9 -1
import sys
sys.version

In [ ]:
% cd ~

# Remove the environment
! if [ -d "nn-nbirds" ]; then rm -rf "nn-nbirds"; fi
# ! pip freeze | xargs pip uninstall -y

# Build the environment 
! git clone https://github.com/zacateras/nn-nbirds.git
% cd ./nn-nbirds
! pip install -r requirements.txt
# Those below fixes the issue with keras @ Colab
!pip install Pillow==4.0.0
!pip install PIL
!pip install image

In [ ]:
import shutil
import os
import urllib
import zipfile

if os.path.exists('data') and os.path.isdir('data'):
    shutil.rmtree('data')
os.mkdir('data')

# Download the dataset
urllib.urlretrieve ('https://www.dropbox.com/s/fi2g3zxsn0pdmn1/nbirds.zip?dl=1', 'data/nbirds.zip')

zip_ref = zipfile.ZipFile('data/nbirds.zip', 'r')
zip_ref.extractall('data')
zip_ref.close()

os.remove('data/nbirds.zip')

In [ ]:
from preprocess import *

# preprocess and load txt ds metadata
ds_meta = build_ds_meta()

In [ ]:
for ds_meta_item in ds_meta:
    print('Quantity of %s: %s' % (ds_meta_item, ds_meta[ds_meta_item].count()[0]))

In [ ]:
# BASE               SET_A
# + Bounding box  => SET_A_BB               (del)
# + Gabor filter  => SET_A_BB_GF            (del)
# + TVT splitting => SET_A_BB_GF_train
#                    SET_A_BB_GF_validation
#                    SET_A_BB_GF_test

apply(bounding_box, 'data/SET_A', 'data/SET_A_BB', ds_meta)
apply(lambda guid, img, dsm: gabor_filter(guid, greyscale(guid, resize(guid, img, dsm), dsm), dsm), 'data/SET_A_BB', 'data/SET_A_BB_GF', ds_meta)
apply_tvt_split('data/SET_A_BB_GF')

In [ ]:
import os

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [ ]:
batch_size = 16

train_labels = os.listdir('data/SET_A_BB_GF_train')

train_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
    'data/SET_A_BB_GF_train',
    target_size=(150, 150),
    batch_size=batch_size,
    classes=train_labels)

model = Sequential()
model.add(Flatten(input_shape=(150, 150, 3))) 
model.add(Dense(256))
model.add(Dense(512))
model.add(Dense(len(train_labels)))
model.add(Activation('sigmoid'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=50)